## DecisionTree

In [1]:
import numpy as np
import operator

def calcShannonEnt(dataSet):#计算信息熵
    numEntries = len(dataSet)
    labelCount = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCount.keys():#统计每一种类别有多少样本
            labelCount[currentLabel] = 0
        labelCount[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCount:
        prob = float(labelCount[key])/numEntries#计算信息熵
        shannonEnt -= prob*(np.log2(prob))
    return shannonEnt


def splitDataSet(dataSet, axis, value):#在axis这一特征上，根据不同的value去划分样本
    retDataSet = []
    for featVec in dataSet:
        if(featVec[axis] == value):
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])#extend是个坑，不能融合数据
            retDataSet.append(reducedFeatVec)
    return retDataSet


def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in np.arange(numFeatures):
        featList = [ example[i] for example in dataSet ]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)#对于每一种特征划分计算信息熵
        infoGain = baseEntropy - newEntropy#计算熵增
        if(infoGain >bestInfoGain ):#找到熵增最大的
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature


def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(),operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]
def createDataSet():
    dataset = [
        [1,1,'yes'],
        [1,1,'yes'],
        [1,0,'no'],
        [0,1,'no'],
        [0,1,'no']
    ]
    labels = ['no surfacing','flippers']
    return dataset,labels

def createTree(dataSet,labels):
    classList = [ example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):#当只有一种类别的时候结束扩展决策树，返回该类别
        return classList[0]
    if len(dataSet[0]) == 1:#当只有一种特征的时候，不能再分的时候，返回数量最多的类别
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels )#采用递归的形式不断的扩展决策树
    return myTree


def storeTree(inputTree, filename):#存储生成的决策树
    import pickle
    fw = open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()


def grabTree(filename):#读取存储的决策树
    import pickle
    fr = open(filename)
    return pickle.load(fr)


def classify(inputTree, featLabels, testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key],featLabels ,testVec)#递归的拆解以字典形式存储的决策树
            else:
                classLabel = secondDict[key]
    return classLabel

if __name__ == "__main__":
   fr = open('lenses.txt')
   lenses = [inst.strip().split('\t') for inst in fr.readlines()]
   lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']
   lensesTree = createTree(lenses, lensesLabels)
   print(lensesTree)

{'tearRate': {'normal': {'astigmatic': {'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'presbyopic': 'no lenses', 'young': 'hard', 'pre': 'no lenses'}}}}, 'no': {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}}}}, 'reduced': 'no lenses'}}
